In [48]:
import json
import networkx as nx
from collections import defaultdict

data = json.load(open("/home/vvejalla/centralized-localization/tcas1_cfg_all_functions.json"))

graphs = []

node_to_lines_all = []

for entry in data:
    G = nx.DiGraph()
    node_to_lines = {}
    # edge from line i to line j iff there is a single block with i and j adjacent in the block 
    # or there are two blocks that are connected and 
    # Add nodes and their line mappings
    
    lines_set = set()
    
    
    for node, info in entry["nodes"].items():
        G.add_node(node)
        lines = info.get("lines", [])
        node_to_lines[node] = lines
        lines_set = lines_set.union(set(lines))

    # Add edges
    for edge in entry["edges"]:
        src, tgt = edge
        G.add_edge(src, tgt)
    
    H = nx.DiGraph()
    H.add_nodes_from(lines_set)
    for node in G.nodes:
        for i in range(1, len(node_to_lines[node])):
            H.add_edge(node_to_lines[node][i-1], node_to_lines[node][i])
    
    for edge in G.edges:
        if len(node_to_lines[edge[0]]) > 0 and len(node_to_lines[edge[1]]) > 0:
            H.add_edge(node_to_lines[edge[0]][-1], node_to_lines[edge[1]][0])
    

    graphs.append(H)
    # node_to_lines_all.append(node_to_lines)

In [49]:
import sys
import os

def parse_file(file):
    lookup = {}
    lines = file.readlines()
    for line in lines:
        splitted = line.strip().split(":", 2)
        if len(splitted) < 3:
            continue
        execs, line_no, statement = splitted
        execs = execs.strip()
        line_no = line_no.strip()
        statement = statement.strip()
        
        if not execs.isnumeric():
            continue
        if not line_no.isnumeric():
            raise Exception('weird line number')
        
        execs = int(execs)
        line_no = int(line_no)
        
        if execs <= 0:
            raise Exception('line executed 0 times')
        
        if line_no in lookup:
            raise Exception('repeated line number')
        
        lookup[line_no] = statement
    
    return lookup

passing_dir = "coverage_output/tcas1_passing"
failing_dir = "coverage_output/tcas1_failing"

counts = {}
statement_lookup = {}

for i,dir in [(0,passing_dir), (1,failing_dir)]:
    for gcov_file in os.listdir(dir):
        file_name = os.path.join(dir, gcov_file)
        file = open(file_name)
        lookup = parse_file(file)
        file.close()
        
        for line_no in lookup:
            if line_no not in counts:
                counts[line_no] = [0,0]
            
            counts[line_no][i] += 1
            
            if line_no in statement_lookup:
                assert statement_lookup[line_no] == lookup[line_no]
            else:
                statement_lookup[line_no] = lookup[line_no]

print(counts)

{46: [1539, 37], 48: [1539, 37], 49: [1539, 37], 50: [1539, 37], 51: [1539, 37], 52: [1539, 37], 54: [546, 30], 56: [546, 30], 59: [847, 37], 64: [847, 37], 70: [847, 37], 71: [847, 37], 79: [847, 37], 82: [847, 37], 88: [847, 37], 89: [847, 37], 97: [847, 37], 105: [592, 37], 107: [592, 37], 110: [1539, 37], 117: [1539, 37], 120: [1539, 37], 131: [847, 37], 133: [710, 37], 136: [588, 10], 139: [1539, 37], 142: [1570, 37], 146: [1570, 37], 155: [1539, 37], 156: [1539, 37], 157: [1539, 37], 158: [1539, 37], 159: [1539, 37], 160: [1539, 37], 161: [1539, 37], 162: [1539, 37], 163: [1539, 37], 164: [1539, 37], 165: [1539, 37], 166: [1539, 37], 167: [1539, 37], 169: [1539, 37], 170: [1539, 37], 100: [535, 27], 102: [535, 27], 122: [153, 0], 126: [137, 0], 132: [137, 0], 134: [122, 27], 148: [31, 0], 149: [31, 0], 150: [31, 0], 151: [31, 0], 152: [31, 0], 153: [31, 0]}


In [55]:
nodes = graphs[3].nodes
mapping = {node: int(node[node.find(":")+1:]) for node in nodes}

In [56]:
mapping

{'tcas1.c:79': 79,
 'tcas1.c:70': 70,
 'tcas1.c:73': 73,
 'tcas1.c:71': 71,
 'tcas1.c:77': 77}

In [57]:
for i in range(len(graphs)):
    nodes = graphs[i].nodes
    mapping = {node: int(node[node.find(":")+1:]) for node in nodes}
    graphs[i] = nx.relabel_nodes(graphs[i], mapping)

In [58]:
graphs[8].nodes

NodeView((157, 163, 153, 149, 150, 164, 166, 165, 159, 146, 160, 152, 148, 167, 169, 170, 161, 151, 156, 158, 162, 155))

In [64]:
for i in range(len(graphs)):
    for node in graphs[i]:
        if node in counts:
            graphs[i].nodes[node]["passing"] = counts[node][0]
            graphs[i].nodes[node]["failing"] = counts[node][1]
        else:
            graphs[i].nodes[node]["passing"] = 0
            graphs[i].nodes[node]["failing"] = 0